<a href="https://colab.research.google.com/github/akankshanehete/RNNtesting/blob/main/Smartphonesensor_Activity_Classification_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import time
from sklearn.preprocessing import RobustScaler
from tensorflow import keras
from keras import preprocessing
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization,Masking
from scipy import stats

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)
tf.random.set_seed(RANDOM_SEED)

In [2]:
#PARAMETERS 
window_size = 15

In [3]:
!gdown --id 152sWECukjvLerrVG2NUO8gtMFg83RKCF --output WISDM_ar_latest.tar.gz
!tar -xvf WISDM_ar_latest.tar.gz

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=152sWECukjvLerrVG2NUO8gtMFg83RKCF
To: /content/WISDM_ar_latest.tar.gz
100% 11.4M/11.4M [00:00<00:00, 74.8MB/s]
WISDM_ar_v1.1/
WISDM_ar_v1.1/readme.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_raw_about.txt
WISDM_ar_v1.1/WISDM_ar_v1.1_transformed.arff
WISDM_ar_v1.1/WISDM_ar_v1.1_trans_about.txt


In [4]:
column_names = ['user_id', 'activity', 'timestamp', 'x_axis', 'y_axis', 'z_axis']

df = pd.read_csv('WISDM_ar_v1.1/WISDM_ar_v1.1_raw.txt', header=None, names=column_names)
df.z_axis.replace(regex=True, inplace=True, to_replace=r';', value=r'')
df['z_axis'] = df.z_axis.astype(np.float64)
df.dropna(axis=0, how='any', inplace=True)

In [5]:
df.head()

,user_id,activity,timestamp,x_axis,y_axis,z_axis
0,33,Jogging,49105962326000,-0.694638,12.680544,0.503953
1,33,Jogging,49106062271000,5.012288,11.264028,0.953424
2,33,Jogging,49106112167000,4.903325,10.882658,-0.081722
3,33,Jogging,49106222305000,-0.612916,18.496431,3.023717
4,33,Jogging,49106332290000,-1.184970,12.108489,7.205164


In [6]:
columns = ['x_axis', 'y_axis', 'z_axis']
x_columns = ['x_axis']
y_columns = ['y_axis']
z_columns = ['z_axis']


In [7]:
df_train = df[df['user_id'] <= 30]
df_test = df[df['user_id'] > 30]

In [8]:
scale_columns = ['x_axis', 'y_axis', 'z_axis']

scaler = RobustScaler()

scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(
  df_train[scale_columns].to_numpy()
)

df_test.loc[:, scale_columns] = scaler.transform(
  df_test[scale_columns].to_numpy()
)

NameError: ignored

In [ ]:
# model constants
BATCH_SIZE = 50
EPOCHS = 10

In [ ]:
# building the rnn model (adding layers)

accuracies = []
runtimes = []
for _ in range (10):
  model = Sequential()
  # add masking layer to the RNN to avoid the padded values in series since they are of variable length
  model.add(tf.keras.layers.Masking(mask_value=-100, input_shape=(seq_len, 6)))
  model.add(LSTM(128, input_shape=(x_train.shape[1:]), return_sequences=True))

  model.add(Dropout(0.2))
  model.add(BatchNormalization())  

  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.1))
  model.add(BatchNormalization())

  model.add(LSTM(128))
  model.add(Dropout(0.2))
  model.add(BatchNormalization())

  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.2))

  # last layer has 2 nodes due to there being binary classification
  model.add(Dense(6, activation='softmax'))

  opt = tf.keras.optimizers.Adam(learning_rate=0.001, decay=1e-6)
    
  model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'])

  start = time.perf_counter()
  # training the model
  model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test,y_test), shuffle=False)
  result = model.evaluate(x_test,y_test)
  elapsed = time.perf_counter() - start
  accuracies.append(result[1])
  runtimes.append(elapsed)
  

mean_accuracy = sum(accuracies)/len(accuracies)
print('mean accuracy: ' + str(mean_accuracy))
print('standard_deviation:' + str(np.std(accuracies)))
mean_runtime = sum(runtimes)/len(runtimes)
print('mean runtime:' + str(mean_runtime))
print('standard deviation:' + str(np.std(runtimes)))

Epoch 1/10
2/2 [==============================] - 31s 12s/step - loss: 1.9934 - accuracy: 0.3118 - val_loss: 1.6847 - val_accuracy: 0.4884
Epoch 2/10
2/2 [==============================] - 13s 8s/step - loss: 1.1017 - accuracy: 0.4946 - val_loss: 1.6084 - val_accuracy: 0.4651
Epoch 3/10
2/2 [==============================] - 13s 8s/step - loss: 0.8523 - accuracy: 0.6774 - val_loss: 1.5517 - val_accuracy: 0.4767
Epoch 4/10
2/2 [==============================] - 13s 8s/step - loss: 0.8054 - accuracy: 0.6237 - val_loss: 1.5112 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 14s 8s/step - loss: 0.7236 - accuracy: 0.7097 - val_loss: 1.4849 - val_accuracy: 0.5000
Epoch 6/10
2/2 [==============================] - 14s 8s/step - loss: 0.6627 - accuracy: 0.7312 - val_loss: 1.4564 - val_accuracy: 0.5233
Epoch 7/10
2/2 [==============================] - 13s 8s/step - loss: 0.5786 - accuracy: 0.7957 - val_loss: 1.4266 - val_accuracy: 0.5349
Epoch 8/10
2/2 [=================